In [ ]:
import pandas as pd

games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")
ranking = pd.read_csv("Data/ranking.csv")
player = pd.read_csv("Data/players.csv")

In [ ]:
bad_games = (
    details
    .groupby("GAME_ID")["START_POSITION"]
    .apply(lambda x: x.notna().sum())
    .loc[lambda x: x != 10]
)
bad_game_ids = bad_games.index
details[details["GAME_ID"].isin(bad_game_ids)]

In [ ]:
# START_POSITION 확인
bad_games = (
    details
    .groupby("GAME_ID")["START_POSITION"]
    .apply(lambda x: x.notna().sum())
    .loc[lambda x: x != 10]
)
bad_games

In [ ]:
details.info()

In [ ]:
details[details["GAME_ID"]==22200477]

In [ ]:
details.describe()

In [ ]:
details.head()

In [ ]:
# GSW의 자료에 null값 확인 -> 전부 부상이나 특정 이유로 불참.
# Null값은 제외하고 생각. 참여자체를 안했기에.
cols_to_check = ["MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT",
                 "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB",
                 "AST", "STL", "BLK", "TO", "PF", "PTS"]
null_condition = details[cols_to_check].isna().any(axis=1)
gsw_nulls = details[null_condition & (details["TEAM_ABBREVIATION"] == "GSW")]
(
    gsw_nulls["COMMENT"]
    .notna()
    & gsw_nulls["COMMENT"].str.strip().ne("")
).all()

## Details 테이블 전처리
### 전처리 필요가 없는 칼럼
- GAME_ID, TEAM_ID,TEAM_ABBREVIATION, PLAYER_ID

### 전처리 필요한 칼럼
- PLAYER_NAME(lower), START_POSITION(카테고리 화, null처리), MIN(min.decimal, null 처리), PLUS_MINUS(Null값 처리), FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT,OREB, DREB, REB, AST, STL, BLK, TO, PF, PTS(Null 처리)

In [ ]:
# clean_details 생성
clean_details = details.copy()

In [ ]:
# 필요한 칼럼들만 선택
clean_details = clean_details[
    ["GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION", "PLAYER_ID", "PLAYER_NAME",
     "START_POSITION", "MIN", "PLUS_MINUS", "FGM", "FGA", "FG_PCT", "FG3M",
     "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB", "AST",
     "STL", "BLK", "TO", "PF", "PTS"]
].copy()

In [ ]:
# Start_position 카테고리화
clean_details["START_POSITION"] = clean_details["START_POSITION"].astype("category")

In [ ]:
# Start_position 처리되었는지 확인
clean_details["START_POSITION"].dtype

In [ ]:
# MIN 처리 전 확인 1
clean_details["MIN"].apply(type).value_counts()

In [ ]:
# MIN 처리 전 확인 2
clean_details[
    clean_details["MIN"].apply(type) == str]["MIN"].value_counts().head(10)

In [ ]:
# MIN 처리 전 확인 3
clean_details["MIN"].value_counts(dropna=False)

In [ ]:
# MIN into float
def minutes_to_decimal(x):
    if pd.isna(x):
        return pd.NA

    if isinstance(x, (int, float)):
        return float(x)

    s = str(x).strip()

    if ":" in s:
        mins, secs = s.split(":")
        return float(mins) + float(secs) / 60

    return float(s)

clean_details["MIN_DECIMAL"] = clean_details["MIN"].apply(minutes_to_decimal)


In [ ]:
# 경기에 참여하지 않은 선수들의 데이터에 대한 자료는 drop
cols_to_check = ["MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT",
                 "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB",
                 "AST", "STL", "BLK", "TO", "PF", "PTS"]
clean_details = clean_details.dropna(subset=cols_to_check, how="all")

In [ ]:
# drop한 결과 확인
before = len(details)
after = len(clean_details)
print(f"{before - after} rows dropped.")

In [ ]:
# PLUS_MINUS 칼럼 확인
clean_details[clean_details["PLUS_MINUS"].isnull()]

In [ ]:
# PLUS_MINUS 칼럼 중에 가능한 것들만 나타내는 파생칼럼 생성.
clean_details["PLUS_MINUS_AVAILABLE"] = clean_details["PLUS_MINUS"].notna()
clean_details["PLUS_MINUS_AVAILABLE"].value_counts()

## games 테이블 전처리
### 전처리 필요가 없는 칼럼
GAME_ID, HOME_TEAM_ID, VISITOR_TEAM_ID
### 전처리 필요한 칼럼
- GAME_DATE_EST(date), PTS_home(Null처리), PTS_away(Null처리), SEASON(Null처리)

In [ ]:
# clean_games 생성
clean_games= games.copy()

In [ ]:
# 필요한 칼럼들만 선택
clean_games = clean_games[
    ["GAME_ID", "GAME_DATE_EST", "HOME_TEAM_ID", "VISITOR_TEAM_ID",
     "PTS_home", "PTS_away", "SEASON"]
]

In [ ]:
clean_games.info()

In [ ]:
# PTS_home 확인 1
clean_games[clean_games["PTS_home"].isnull()]

In [ ]:
# PTS_home 확인 2
bad_game_ids = games[
    clean_games.filter(like="_home").isna().all(axis=1)
]["GAME_ID"]

clean_details[clean_details["GAME_ID"].isin(bad_game_ids)].shape

In [ ]:
# PTS_home, PTS_away, SEASON null인 것들을 drop
# 경기자체의 데이터가 없는 빈껍질 스케줄이기 때문에 drop
clean_games = clean_games.dropna()

In [ ]:
# drop 결과 확인
before = len(games)
after = len(clean_games)
print(f"{before - after} rows dropped.")